In [1]:
from gssutils import *

In [2]:
if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    boo_pythonNB_environment = True
else:
    boo_pythonNB_environment = False

print('Execution Environment: ' + get_ipython().__class__.__name__ + '; setting boo_pythonNB_environment to: ' \
      + str(boo_pythonNB_environment))

Execution Environment: ZMQInteractiveShell; setting boo_pythonNB_environment to: True


In [3]:
from gssutils import *

#if boo_pythonNB_environment == False:
scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/' \
                  'healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk')
scraper

#else:
#    tabs = loadxlstabs('/Users/martyn/Downloads/hslepivotab1.xlsx') # Not significantly quicker with local read.
# Download not the bottle-neck of the 50Mb file - read to memory is what is taking the time.    


## Health state life expectancy, all ages, UK

Pivot tables for health state life expectancy by sex and area type, divided by three-year intervals starting from 2009 to 2011.

### Description

Pivot tables for health state life expectancy by sex and area type, divided by two-year intervals starting from 2009 to 2011.

### Distributions

1. Health state life expectancy, all ages, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/heestimates.xlsx)) - 2016-11-29
1. Health state life expectancy, all ages, UK ([application/zip](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v1/healthexpectanciespivottables.zip)) - 2016-11-29
1. Health state life expectancy, all ages, UK ([application/zip](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v2/healthexpectanciespivottablesversion2.zip)) - 2016-11-29
1. Health state life expectancy, all ages, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v3/refpivottablesfinal.xlsx)) - 2016-11-29
1. Health state life expectancy, all ages, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v4/hslepivotab1.xlsx)) - 2016-11-29


In [4]:
tabs = scraper.distributions[0].as_databaker()
distribution = scraper.distributions[0]
display(distribution)

Spreadsheet is 50Mb - note to developers - should we cut down the spreadsheet by deleting hidden columns etc. before processing? Each TAB is also calculated from pivot(s); the RAW data being hidden.

In [5]:
str_tabsheetsinfocus = 'H'
if boo_pythonNB_environment == True:
    print(str_tabsheetsinfocus)

H


More condensed code version of above... only one needed...

In [6]:
i = 0
pd_df_name = []
pd_df_cleaned = []
import numpy as np
for tab in tabs:
    if tabs[i].name.startswith(str_tabsheetsinfocus):
        
        print(' Scanning: ' + tabs[i].name)

        try:
                pd_df_name.append(tabs[i].name)
                pd_tab = distribution.as_pandas(sheet_name = tabs[i].name) #, skiprows=1, header=None)
                
                if tabs[i].name == 'HE - Country level estimates':
                    pd_df_dimensions = pd_tab.iloc[:, :7]
                    pd_df_observations = pd_tab.iloc[:, 7:14]
                    pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                    
                if tabs[i].name == 'HE - Region level estimates':
                    pd_df_dimensions = pd_tab.iloc[:, :8]
                    pd_df_observations = pd_tab.iloc[:, 8:14]
                    pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                    pd_df_original.columns = pd_df_original.iloc[0]
                    pd_df_original = pd_df_original[1:]
                    
                if tabs[i].name == 'HE - MC,CA,WHB estimates':
                    pd_df_dimensions = pd_tab.iloc[:, :7]
                    pd_df_observations = pd_tab.iloc[:, 7:14]
                    pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                    pd_df_original.columns = pd_df_original.iloc[0]
                    pd_df_original = pd_df_original[1:]
                
                if tabs[i].name == 'HE - Local area estimates':
                    pd_df_dimensions = pd_tab.iloc[:, :8]
                    pd_df_observations = pd_tab.iloc[:, 8:15]
                    pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                    pd_df_original.columns = pd_df_original.iloc[0]
                    pd_df_original = pd_df_original[1:]
                
                pd_df_original['Sex'] = np.where(pd_df_original['Sex'] == 'Males', 'M', pd_df_original['Sex'])
                pd_df_original['Sex'] = np.where(pd_df_original['Sex'] == 'Females', 'F', pd_df_original['Sex'])
                pd_df_original['Period'] = pd_df_original['Period'].map(lambda x: f'gregorian-interval/{str(x)[:4]}-03-31T00:00:00/P2Y')
                #t[prd] = t[prd].map(lambda x: 'government-year/' + left(x,4) +'-20' + right(x,2))
                pd_df_cleaned.append(pd_df_original.dropna(how='all'))    
                print('Processed: ' + tabs[i].name)
                
        except ERR_HECountryLevelEstimates:
                print('Error within ' + str(pd_df_name.append(tabs[i].name)) + ' process to extract to Pandas DF.')

        
    i += 1
    

 Scanning: HE - Country level estimates
Processed: HE - Country level estimates
 Scanning: HE - Region level estimates
Processed: HE - Region level estimates
 Scanning: HE - MC,CA,WHB estimates
Processed: HE - MC,CA,WHB estimates
 Scanning: HE - Local area estimates
Processed: HE - Local area estimates


In [7]:
del pd_df_dimensions, pd_df_observations, pd_df_original, pd_df_name

In [8]:
#This code is for testing purposes - not for final.
#if boo_pythonNB_environment == True:
    #for i in range(len(pd_df_name)):
        #print('-------------------------------------------------------------------')
        #print(pd_df_name[i])
        #print('-------------------------------------------------------------------')
        #display(pd_df_cleaned[i].head(3))
        #print('Record Count: ' + str(pd_df_cleaned[i].shape[0]))
        #print(' ')

In [9]:
#pd_df_cleaned[0]

In [10]:
#### Drop some column headings:
try:
    pd_df_cleaned[0] = pd_df_cleaned[0].drop(columns=['Country', 'sex1', 'ageband'])
except Exception as e:
    print("1. " + str(e))
try:
    pd_df_cleaned[1] = pd_df_cleaned[1].drop(columns=['Area_name', 'sex1', 'ageband'])
except Exception as e:
    print("2. " + str(e))
try:
    pd_df_cleaned[2] = pd_df_cleaned[2].drop(columns=['Area', 'sex1', 'ageband'])
except Exception as e:
    print("3. " + str(e))
try:
    pd_df_cleaned[3] = pd_df_cleaned[3].drop(columns=['Country', 'Area_name', 'sex1', 'ageband'])
except Exception as e:
    print("4. " + str(e))

## Leigh - nothing below is ready - just random code snippets that can be deleted....

In [11]:
#### Rename some of the columns removing some characters (,),_,%,"
i = 0
for t in pd_df_cleaned:
    cols = pd.DataFrame(list(t))
    cols.columns = ['ColumnNames']
    cols = pd.DataFrame(cols['ColumnNames']
                    .str.replace('(','')
                    .str.replace(')','')
                    .str.replace('_','')
                    .str.replace('%','')
                    .str.replace('''"''','')
                    .str.strip()
                    )
    #### Rename all the columns
    t.columns = [cols.iloc[:, 0].tolist()]


In [12]:
#### Rename some columns and change some values to match standard formats
#### Then split each dataset into 3, rename so they all match and then concatenate
import numpy as np
genHead = "Gender"
prd = "Period"
ageRng = 'ONS Age Range'
newColNme = "Life Expectancy Estimate Type"
lciColNme = 'Lower CI'
uciColNme = 'Upper CI'
onsGeog = 'ONS Geography'
i = 0
for t in pd_df_cleaned:
    try:
        t = t.rename(columns={'Sex':genHead,'Code':onsGeog,'age group':ageRng,'Age group':ageRng})
    except Exception as e:
        print(f"Renames: {i}. " + str(e))
        
    try:
        #### Split the table into 3, reconfigure some column nsmes and then concatenate
        tbl1 = t[[prd,onsGeog,genHead,ageRng,'Life Expectancy LE','LE Lower CI','LE Upper CI']]
        tbl2 = t[[prd,onsGeog,genHead,ageRng,'Healthy Life Expectancy HLE','HLE Lower CI','HLE Upper CI']]
        tbl3 = t[[prd,onsGeog,genHead,ageRng,'Proportion of Life Spent in Good Health']]
        #### Rename some columns so all 3 tables match ready for joining
        tbl1[newColNme] = 'LE'
        tbl1 = tbl1.rename(columns={'Life Expectancy LE':'Value','LE Lower CI':lciColNme,'LE Upper CI':uciColNme})
        tbl2[newColNme] = 'HLE'
        tbl2 = tbl2.rename(columns={'Healthy Life Expectancy HLE':'Value','HLE Lower CI':lciColNme,'HLE Upper CI':uciColNme})
        tbl3[lciColNme] = ''
        tbl3[uciColNme] = ''
        tbl3[newColNme] = 'PLGH'
        tbl3 = tbl3.rename(columns={'Proportion of Life Spent in Good Health':'Value'})

        newTbl = pd.concat([tbl1,tbl2,tbl3])
        pd_df_cleaned[i] = newTbl[[prd,onsGeog,genHead,ageRng,newColNme,lciColNme,uciColNme,'Value']]
        #break
    except Exception as e:
        print(f"Table Split: {i}. " + str(e))
    i = i + 1


/Users/leigh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/leigh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/leigh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [13]:
#### Set up the folder path for the output files
from pathlib import Path

out = Path('out')
out.mkdir(exist_ok=True, parents=True)

In [14]:
headSet = [
    'HE Country level estimates',
    'HE Region level estimates',
    'HE MC CA WHB estimates',
    'HE Local area estimates'
]
headMain = 'Pivot tables for health state life expectancy by sex and area type, - HE Country level estimates divided by two-year intervals starting from 2009 to 2011.'

In [15]:
i = 0
for t in pd_df_cleaned:
    try:
        fleNme =  headSet[i].replace(' ','-').lower() + '.csv'
        hed =  headSet[i].replace(' ','-').lower()
        t =  t.drop_duplicates()
        t.to_csv(out / fleNme, index = False)
    
        scraper.set_dataset_id(f'gss_data/disability/ons-health-state-life-expectancy/{hed}/')
    
        scraper.dataset.family = 'disability'
    
        with open(out / ('pre' + fleNme + '-metadata.trig'), 'wb') as metadata:metadata.write(scraper.generate_trig())

        csvw = CSVWMetadata('https://gss-cogs.github.io/family-disability/reference/')
        csvw.create(out / fleNme, out / (fleNme + '-schema.json'))
        i = i + 1
        #break
    except Exception as e:
        print(f"Table Split: {i}. " + str(e))

"Gender" not defined
"Age Group" not defined
"Estimate Type" not defined
"Lower CI" not defined
"Upper CI" not defined
"Gender" not defined
"Age Group" not defined
"Estimate Type" not defined
"Lower CI" not defined
"Upper CI" not defined
"Gender" not defined
"Age Group" not defined
"Estimate Type" not defined
"Lower CI" not defined
"Upper CI" not defined
"Gender" not defined
"Age Group" not defined
"Estimate Type" not defined
"Lower CI" not defined
"Upper CI" not defined


In [16]:
#### As each trig file is created multiple @prefix ns lines are added. This code gets rid of them

import os
i = 1 #### Main looping index
k = 1 #### Secondary index to skip over lines with ns2
lineWanted = False
#### Loop around each element in the main heading list
for t in headSet:
    hed = t.replace(' ','-').lower()
    newDat = ''
    curNme = f'out/pre{hed}.csv-metadata.trig'    #### Current file name
    newNme = f'out/{hed}.csv-metadata.trig'       #### New file name
    #### Open the file and loop around each line adding or deleting as you go
    with open(curNme, "r") as input:
        #### Also open the new file to add to as you go
        with open(newNme, "w") as output: 
            #### Loop around the input file
            for line in input:
                #### Change the lines to the value in the variabl headMain
                if headMain in line.strip("\n"):
                    newLine = line
                    newLine = line.replace(headMain, headMain + ' - ' + t)
                    output.write(newLine)
                else: 
                    lineWanted = True
                    #### Ignore lines with ns2 but loop for other ns# lines, deleteing any extra ones that do not match the value of k
                    if '@prefix ns2:' not in line.strip("\n"):
                        if '@prefix ns' in line.strip("\n"):
                            if f'@prefix ns{k}:' not in line.strip("\n"):
                                #### You do not want this line so ignore
                                lineWanted = False
                    #### If the line is needed check if it is a line that needs changing then write to new file 
                    if lineWanted: 
                        if 'a pmd:Dataset' in line.strip("\n"):
                            line = line.replace(f'{hed}/', f'{hed}')
                    
                        if 'pmd:graph' in line.strip("\n"):
                            line = line.replace(f'{hed}/', f'{hed}')
                        #### Output the line to the new file                    
                        output.write(line)
                        
    #### Close both files
    input.close
    output.close
    #### Old trig file no longer needed so remove/delete
    os.remove(curNme)

    #### Increment i, ns2 is used for something else so you have got to jump k up by 1 at this point
    i = i + 1
    if i == 2:
        k = k + 2
    else:
        k = k + 1


In [17]:
#### Notes - amendments link to cards? was 50mb now 26